In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
client = MongoClient('localhost', 27017)
collection = client['clean2'].clean2

In [2]:
db = pd.DataFrame(list(collection.find()))
db.head()
    
  

,_id,category_code,founded_year,geo,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised
0,5cd590c53c474a367c3cc350,software,2008,"{'type': 'Point', 'coordinates': [-122.392142,...",37.782263,-122.392142,GoingOn,40.0,950000.0,USD,$8.5M
1,5cd590c53c474a367c3cc351,web,2011,"{'type': 'Point', 'coordinates': [-95.712891, ...",37.090240,-95.712891,Mokitown,NaN,NaN,None,$0
2,5cd590c53c474a367c3cc352,games_video,2008,"{'type': 'Point', 'coordinates': [-122.151158,...",47.597965,-122.151158,YouBeQB,8.0,NaN,None,$0
3,5cd590c53c474a367c3cc353,games_video,2008,"{'type': 'Point', 'coordinates': [-122.393229,...",37.781265,-122.393229,Crunchyroll,50.0,4050000.0,USD,$4.8M
4,5cd590c53c474a367c3cc354,web,2013,"{'type': 'Point', 'coordinates': [-122.323895,...",37.566879,-122.323895,Fixya,30.0,2000000.0,USD,$8M


In [3]:
def compañias_cercanas(geo, max_distance):
    cercanas = pd.DataFrame(list(collection.find(
        {'geo': { '$near': {'$geometry': geo, '$maxDistance': max_distance}}}, {'name': 1})))
    return len(cercanas)

In [4]:
distancia = 3000
db['numero_compañias_cerca'] = db['geo'].apply(lambda x: compañias_cercanas(x,distancia))

In [5]:

db.head()

,_id,category_code,founded_year,geo,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised,numero_compañias_cerca
0,5cd590c53c474a367c3cc350,software,2008,"{'type': 'Point', 'coordinates': [-122.392142,...",37.782263,-122.392142,GoingOn,40.0,950000.0,USD,$8.5M,38
1,5cd590c53c474a367c3cc351,web,2011,"{'type': 'Point', 'coordinates': [-95.712891, ...",37.090240,-95.712891,Mokitown,NaN,NaN,None,$0,4
2,5cd590c53c474a367c3cc352,games_video,2008,"{'type': 'Point', 'coordinates': [-122.151158,...",47.597965,-122.151158,YouBeQB,8.0,NaN,None,$0,2
3,5cd590c53c474a367c3cc353,games_video,2008,"{'type': 'Point', 'coordinates': [-122.393229,...",37.781265,-122.393229,Crunchyroll,50.0,4050000.0,USD,$4.8M,38
4,5cd590c53c474a367c3cc354,web,2013,"{'type': 'Point', 'coordinates': [-122.323895,...",37.566879,-122.323895,Fixya,30.0,2000000.0,USD,$8M,2


In [6]:
db = db.drop(db[db['numero_compañias_cerca']<25].index)
db.head()

,_id,category_code,founded_year,geo,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised,numero_compañias_cerca
0,5cd590c53c474a367c3cc350,software,2008,"{'type': 'Point', 'coordinates': [-122.392142,...",37.782263,-122.392142,GoingOn,40.0,950000.0,USD,$8.5M,38
3,5cd590c53c474a367c3cc353,games_video,2008,"{'type': 'Point', 'coordinates': [-122.393229,...",37.781265,-122.393229,Crunchyroll,50.0,4050000.0,USD,$4.8M,38
7,5cd590c53c474a367c3cc357,web,2008,"{'type': 'Point', 'coordinates': [-73.985433, ...",40.740231,-73.985433,BeliefNet,NaN,7000000.0,USD,$7M,38
13,5cd590c53c474a367c3cc35d,web,2008,"{'type': 'Point', 'coordinates': [-122.404488,...",37.790032,-122.404488,Unknown Vector,7.0,NaN,None,$0,38
23,5cd590c53c474a367c3cc367,web,2008,"{'type': 'Point', 'coordinates': [-73.985281, ...",40.723652,-73.985281,Kluster,NaN,1000000.0,USD,$1M,31


In [7]:
db.to_csv('tableau_project_parte2.csv')